In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [4]:
X_train,X_test,Y_train,Y_test=train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [5]:
scaller=StandardScaler()
X_train=scaller.fit_transform(X_train)
X_test=scaller.transform(X_test)

In [6]:
encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_train)
Y_test=encoder.transform(Y_test)

In [8]:
x_test_tensor=torch.from_numpy(X_test.astype(np.float32))
y_test_tensor=torch.from_numpy(Y_test.astype(np.float32))
x_train_tensor=torch.from_numpy(X_train.astype(np.float32))
y_train_tensor=torch.from_numpy(Y_train.astype(np.float32))

In [9]:
class customDataset(Dataset):
    def __init__(self,features,labels):
        self.features=features
        self.labels=labels
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self,index):
        return self.features[index],self.labels[index]

In [17]:
 class SimpleNN(nn.Module):
     def __init__(self,num_features):
         super().__init__()
         self.linear=nn.Linear(num_features,1)
         self.sigmoid=nn.Sigmoid()
     def forward(self,features):
         out = self.linear(features)
         out = self.sigmoid(out)
         return out

         

In [18]:
lr=0.1
epochs=25

In [32]:
train_dataset = customDataset(x_train_tensor,y_train_tensor)
test_dataset= customDataset(x_test_tensor,y_test_tensor)

In [33]:
train_dataLoader = DataLoader(train_dataset,batch_size=10,shuffle=True)
test_dataLoader = DataLoader(test_dataset,batch_size=10,shuffle=True)

In [34]:
loss_function = nn.BCELoss()

In [35]:
model=SimpleNN(x_train_tensor.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=lr)

In [36]:
for epoch in range(epochs):
    for batch_feature,batch_label in train_dataLoader:
            ypred = model(batch_feature)
            loss=loss_function(ypred,batch_label.view(-1,1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f'for epoch:{epoch+1} loss is {loss.item()}')

for epoch:1 loss is 0.7785276174545288
for epoch:1 loss is 0.5620957612991333
for epoch:1 loss is 0.4637605547904968
for epoch:1 loss is 0.3182588219642639
for epoch:1 loss is 0.18888680636882782
for epoch:1 loss is 0.3876829147338867
for epoch:1 loss is 0.3022252023220062
for epoch:1 loss is 0.38600391149520874
for epoch:1 loss is 0.26406532526016235
for epoch:1 loss is 0.12841184437274933
for epoch:1 loss is 0.20654217898845673
for epoch:1 loss is 0.15012992918491364
for epoch:1 loss is 0.2615880072116852
for epoch:1 loss is 0.12064290046691895
for epoch:1 loss is 0.14991988241672516
for epoch:1 loss is 0.12738226354122162
for epoch:1 loss is 0.45336681604385376
for epoch:1 loss is 0.1358659565448761
for epoch:1 loss is 0.12489096820354462
for epoch:1 loss is 0.27429550886154175
for epoch:1 loss is 0.1635981798171997
for epoch:1 loss is 0.22341449558734894
for epoch:1 loss is 0.12554685771465302
for epoch:1 loss is 0.158113494515419
for epoch:1 loss is 0.2505558729171753
for epoch:1 

In [38]:
model.eval()
model_accuracy=[]
with torch.no_grad():
    for batch_features,batch_labels in test_dataLoader:
        ypred=model(batch_features)
        ypred=(ypred>0.5).float()
        accuracy=(ypred==batch_labels).float().mean().item()
        model.append(accuracy)

0.5799999833106995
1.0
0.5
0.6800000071525574
0.5799999833106995
0.6800000071525574
0.5199999809265137
0.5199999809265137
0.5199999809265137
0.5400000214576721
0.6800000071525574
0.625
